# 1.Import package

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data, DataLoader
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
from torch.nn import BatchNorm1d
from torch.utils.data import Dataset
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import global_add_pool, global_mean_pool
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean
from torch.optim.lr_scheduler import ReduceLROnPlateau
import deepchem as dc
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


# 2. Data processing

In [2]:
import sys
import pandas as pd
sys.path.append('GNN_dev')
from Preprocessor import data_preprocessing
from Featurizer import MoleculeDataset
data_dir = '/home/labhhc/Documents/Workspace/Publication/Y2023/Conference/KSE-BCB2023-GNN-ALK/Data'
raw_dir = data_dir +'/raw'
df = pd.read_csv(f'{data_dir}/Data.csv', index_col=0)
df['ID'] = df.index.astype('object')
df.head(2)

Torch version: 2.0.0.dev20230215+rocm5.3
Cuda available: True
Torch geometric version: 2.3.0


,pChEMBL,Canomicalsmiles,ID
0,10.000000,CCc1cc2c(=O)c3c4ccc(C#N)cc4[nH]c3n(C(C)C)c2cc1...,0
1,9.958607,CCc1cc2c(=O)c3c4ccc(C#N)cc4[nH]c3n(C(C)C)c2cc1...,1


In [3]:
## Processing
process = data_preprocessing(data=df, smiles_col='Canomicalsmiles', ID_col='ID', 
                             target_col='pChEMBL', thresh=7, raw_dir = raw_dir)
process.fit()
process.data_train.head()

,ID,Canomicalsmiles,pChEMBL
0,635,COc1cccc(CNC(=O)c2csc(CNC(=O)c3cccnc3Oc3ccc(Cl...,0
1,1319,COc1cc(N2CCN(C)CC2)ccc1Nc1ncc(Cl)c(Nc2ccccc2S(...,0
2,651,Cc1onc(-c2ccccc2)c1CNC(=O)c1csc(CNC(=O)COc2ccc...,0
3,542,O=C(COc1ccc(OC(F)(F)F)cc1)NCc1nc(C(=O)NCCc2ccc...,0
4,1419,COc1ccc(Nc2ccnc(Nc3ccc(OC)c(OCCN4CCCC4)c3)n2)c...,0


In [4]:
# Featurizer
train_dataset = MoleculeDataset(root = data_dir,
                                filename = "train.csv")
test_dataset = MoleculeDataset(root = data_dir,
                                filename = "test.csv", test = True)

valid_dataset = MoleculeDataset(root = data_dir,
                                filename = "valid.csv", valid = True)

# 3. Training

In [5]:
from model_seed import seed_everything
from GNN_architecture import GNN
from TPE_optimize import suggest_hyperparameters, count_parameters, save_model, objective
seed_everything(42)
def Object(trial):
    objective(trial=trial,train_dataset=train_dataset, valid_dataset =valid_dataset, epochs =20)

/home/labhhc/anaconda3/envs/PyG_gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.trial import TrialState
#if __name__ == "__main__":
seed_everything(42)
sampler = TPESampler(seed = 42)
pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=20)
study = optuna.create_study(direction="minimize",pruner = pruner,sampler=sampler)


#study.optimize(objective, n_trials=2)
study.optimize(Object, n_trials=2)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# 4. Validation

In [6]:
from sklearn.metrics import f1_score, average_precision_score
import mlflow.pytorch
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec
import json
with open('Model/params.json','r') as f:
    best_params =  json.load(f)
model_path = './Model/GNN19.pth'

## 4.1. Internal validation

In [7]:
X_train = process.data_train.drop(['pChEMBL'], axis =1)
y_train = process.data_train['pChEMBL']

In [8]:
from cross_validation import cross_val_score
record = cross_val_score(train_dataset, X_train, y_train, best_params)

0it [00:08, ?it/s]

KeyboardInterrupt



## 4.2. External validation

In [8]:
device = 'cpu'
model = GNN(feature_size = 30, model_params = best_params).to(device)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
params = best_params
all_labels_ex = []
probas_val_ex = []
all_probas_ex = []
all_preds_ex = []
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
with torch.no_grad():
    for _, batch in enumerate(test_loader):
        batch.to(device)
        output_ex = model(batch.node_features.float(),batch.edge_attr.float(),
                           batch.edge_index,batch.batch,params).to(device)

        all_probas_ex.append(torch.sigmoid(output_ex).cpu().detach().numpy())
        all_preds_ex.append(np.rint(torch.sigmoid(output_ex).cpu().detach().numpy()))
        all_labels_ex.append(batch.y.cpu().detach().numpy())

all_preds_ex = np.concatenate(all_preds_ex).ravel()
all_labels_ex = np.concatenate(all_labels_ex).ravel()
all_probas_ex = np.concatenate(all_probas_ex).ravel()        
#loss_val = validation_loss/len(valid_loader)
f1_ex = f1_score(all_labels_ex,all_preds_ex)
ap_ex = average_precision_score(all_labels_ex,all_probas_ex)
mlflow.log_metric(key="f1-ex", value=float(f1_ex))
mlflow.log_metric(key="AP-ex", value=float(ap_ex))

print(classification_report(all_labels_ex,all_preds_ex))
print(f1_score(all_labels_ex,all_preds_ex))
print(average_precision_score(all_labels_ex,all_probas_ex))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       237
           1       0.87      0.85      0.86        96

    accuracy                           0.92       333
   macro avg       0.91      0.90      0.90       333
weighted avg       0.92      0.92      0.92       333

0.8631578947368421
0.938043433027809
